## Final Project Submission

Please fill out:
* Student name: Kevin Spring
* Student pace: self paced
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [ ]:
# Your code here - remember to use markdown cells for comments as well!

### Import Data

The data used in this analysis are four files provided by Flatiron for analysis.

- IMDB (SQLite Database)
- Box Office Mojo